# 1. Modern Portfolio Theory

## 1.1 Import Libraries

In [1]:
# Import pandas & yfinance
import pandas as pd
import yfinance as yf
# Import numpy
import numpy as np
from numpy.linalg import multi_dot
# Set numpy random seed
np.random.seed(23)
# Import cufflinks
import cufflinks as cf
cf.set_config_file(offline=True, dimensions=((1000,600)))
# Import plotly express for EF plot
import plotly.express as px
px.defaults.width, px.defaults.height = 1000, 600
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)

## 1.2 Retrive Data

In [2]:
# Specify assets / stocks
# international etf portfolio : 'SPY', 'GLD', 'IWM', 'VWO', 'BND']
# indian stocks : bank, consumer goods, diversified, it, consumer durables
# ['HDFCBANK', 'ITC', 'RELIANCE', 'TCS', 'ASIANPAINT']
assets = ['HDFCBANK', 'ITC', 'RELIANCE', 'TCS', 'ASIANPAINT']
assets.sort()
# Number of assets
numofasset = len(assets)
# Number of portfolio for optimization
numofportfolio = 5000

In [3]:
# Get yahoo tickers for indian stocks
#yahooticker = [x+'.NS' for x in assets]
# Fetch read data for multiple stocks at once
#df = yf.download(yahooticker, start='2015-01-01', end='2022-12-31', progress=False)['Adj Close'] # df.columns = assets
# write data to file for future use
#df.to_csv('/Users/baobach/CQF_notes/pythonlab/03_portfolio_optimization/india_stocks.csv')
# Read from file
df = pd.read_csv('/Users/baobach/CQF_notes/pythonlab/03_portfolio_optimization/india_stocks.csv', index_col=0, parse_dates=True)
# Display dataframe
df.head()

,ASIANPAINT.NS,HDFCBANK.NS,ITC.NS,RELIANCE.NS,TCS.NS
Date,,,,,
2015-01-01,704.9195,446.3565,191.1795,417.9889,1065.1384
2015-01-02,729.2902,452.5686,191.7784,416.8826,1079.3241
2015-01-05,729.2433,448.7476,192.5334,412.3162,1062.9211
2015-01-06,711.8354,441.7619,187.5866,393.6035,1023.7347
2015-01-07,726.1379,443.0512,184.0979,402.1713,1011.6423


## 1.3 Visualize Time Series

In [4]:
# Plot price history
df['2022':].normalize().iplot(kind='line')

In [5]:
# Dataframe of returns and volatility
returns = df.pct_change().dropna()
annual_returns = round(returns.mean()*260*100,2)
annual_stdev = round(returns.std()*np.sqrt(260)*100,2)
df1 = pd.DataFrame({
    'Ann Ret': annual_returns,
    'Ann Vol': annual_stdev
})

In [6]:
# Plot annualized return and volatility
df1.iplot(
    kind='bar',
    shared_xaxes=True,
    orientation='h'
)

## 1.4 Portfolio Composition

In [7]:
df1.reset_index().iplot(
    kind='pie',
    labels='index',
    values='Ann Ret',
    textinfo='percent+label',
    hole=0.4
)

## 1.5 Portfolio Statistics

\\ for explaining the code

### 1.5.1 Portfolio Simulation

In [9]:
def portfolio_simulation(returns): 
    # Initialize the lists
    rets = []; vols = []; wts = [] 
    # Simulate 5,000 portfolios
    for i in range (numofportfolio): 
        # Generate random weights
        weights = np.random.random(numofasset)
    # Set weights such that sum of weights equals 1
        weights /= np.sum(weights)
        # Portfolio statistics
        rets.append(weights.T @ np.array(returns.mean() * 260))
        vols.append(np.sqrt(multi_dot([weights.T, returns.cov()*260, weights])))
        wts.append(weights)
        # Create a dataframe for analysis
        data = {'port_rets': rets, 'port_vols': vols}
        for counter, symbol in enumerate(returns.columns.tolist ()) :
            data[symbol+' weight'] = [w[counter] for w in wts]
            portdf = pd.DataFrame(data)
            portdf['sharpe_ratio'] = portdf['port_rets'] / portdf['port_vols']
            return round(portdf,4)

### 1.5.2 Maximum Sharpe Portfolio

In [10]:
# Create a dataframe for analysis
temp = portfolio_simulation(returns)
temp.head()

,port_rets,port_vols,ASIANPAINT.NS weight,sharpe_ratio
0,0.1848,0.176,0.1893,1.0498


In [11]:
# Get the max sharpe portfolio stats
temp.iloc[temp.sharpe_ratio.idxmax()]

port_rets               0.1848
port_vols               0.1760
ASIANPAINT.NS weight    0.1893
sharpe_ratio            1.0498
Name: 0, dtype: float64

In [12]:
# Verify the above result
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
port_rets,1.0,0.1848,NaN,0.1848,0.1848,0.1848,0.1848,0.1848
port_vols,1.0,0.1760,NaN,0.1760,0.1760,0.1760,0.1760,0.1760
ASIANPAINT.NS weight,1.0,0.1893,NaN,0.1893,0.1893,0.1893,0.1893,0.1893
sharpe_ratio,1.0,1.0498,NaN,1.0498,1.0498,1.0498,1.0498,1.0498


### 1.5.3 Visulaize Simulated Portfolio

In [13]:
# Plot simulated portfolio
fig = px.scatter(
    temp, x='port_vols', y='port_rets', color='sharpe_ratio',
    labels={'port_vols': 'Expected Volatility', 'port_rets': 'Expected Return','sharpe_ratio': 'Sharpe Ratio'}, title="Monte Carlo Simulated Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))
# Plot max sharpe
fig.add_scatter(
    mode='markers',
    x=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_vols']],
    y=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_rets']],
    marker=dict(color='RoyalBlue', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)
# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

## 1.6 Efficient Frontier

### 1.6.1 Constrained Optimization

In [14]:
# Import optimization module from scipy 
# sco.minimize?
import scipy.optimize as sco

### 1.6.2 Portfolio Statistics

In [15]:
def portfolio_stats(weights):
    weights = np.array(weights)
    port_rets = weights.T @ np.array(returns.mean() * 260)
    port_vols = np.sqrt(multi_dot([weights.T, returns.cov() * 260, weights]))
    return np.array([port_rets, port_vols, port_rets/port_vols])
# Minimize the volatility
def min_volatility(weights):
    return portfolio_stats(weights)[1]
# Minimize the variance
def min_variance(weights):
    return portfolio_stats(weights)[1]**2
# Maximizing sharpe ratio
def max_sharpe_ratio(weights):
    return -portfolio_stats(weights)[2]

### 1.6.3 Efficient Frontier Portfolio

In [16]:
# Specify constraints, bounds and initial weights
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bnds = tuple((0,1) for x in range(numofasset))
initial_wts = numofasset*[1./numofasset]

In [17]:
# Optimizing for maximum sharpe ratio
opt_sharpe = sco.minimize(max_sharpe_ratio, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)
# Optimizing for minimum variance
opt_var = sco.minimize(min_variance, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)

In [18]:
opt_sharpe

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1.2178093437864905
       x: [ 2.738e-01  2.292e-01  0.000e+00  2.793e-01  2.176e-01]
     nit: 6
     jac: [-3.827e-04  1.388e-04  3.090e-02  3.888e-05  2.855e-04]
    nfev: 36
    njev: 6

In [19]:
opt_var

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.028040078668510614
       x: [ 1.928e-01  2.367e-01  2.099e-01  7.286e-02  2.878e-01]
     nit: 7
     jac: [ 5.615e-02  5.592e-02  5.597e-02  5.613e-02  5.623e-02]
    nfev: 42
    njev: 7

In [22]:
# Efficient Frontier
targetrets = np.linspace(0.155,0.24,100)
tvols = []
for tr in targetrets:
    ef_cons = ({'type': 'eq', 'fun': lambda x: portfolio_stats(x)[0] - tr},
               {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    opt_ef = sco.minimize(min_volatility, initial_wts, method='SLSQP', bounds=bnds, constraints=ef_cons) 
    tvols.append(opt_ef['fun'])
targetvols = np.array(tvols)

In [23]:
# Create EF Dataframe for plotting
efport = pd.DataFrame({
    'targetrets' : np.around(100*targetrets,2),
    'targetvols': np.around(100*targetvols,2),
    'targetsharpe': np.around(targetrets/targetvols,2)
    })
efport.head()

,targetrets,targetvols,targetsharpe
0,15.50,17.79,0.87
1,15.59,17.72,0.88
2,15.67,17.65,0.89
3,15.76,17.59,0.90
4,15.84,17.52,0.90


In [24]:
# Plot efficient frontier portfolio
fig = px.scatter(
    efport, x='targetvols', y='targetrets',  color='targetsharpe',
    labels={'targetrets': 'Expected Return', 'targetvols': 'Expected Volatility','targetsharpe': 'Sharpe Ratio'}, title="Efficient Frontier Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))
# Plot maximum sharpe portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_sharpe['x'])[1]],
    y=[100*portfolio_stats(opt_sharpe['x'])[0]],
    marker=dict(color='red', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)
# Plot minimum variance portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_var['x'])[1]],
    y=[100*portfolio_stats(opt_var['x'])[0]],
    marker=dict(color='green', size=20, symbol='star'),
    name = 'Min Variance'
).update(layout_showlegend=False)
# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

## 2. References

* [Numpy Linear Algebra](https://numpy.org/doc/stable/reference/routines.linalg.html)
* [Python Resources](https://github.com/kannansingaravelu/PythonResources)
* [Scipy Optimization](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)
* [Styling Plotly Express Figures](https://plotly.com/python/styling-plotly-express)
* [YFinance Documentation](https://github.com/ranaroussi/yfinance)